In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np

from train_utils import *
from breeds import *

## Load indexes

In [4]:
import numpy as np

scores = np.load('breeds_forg.npy')

In [5]:
aver_random_acc_list = []
aver_max_ind_ch_T_acc_list = []
aver_random_ind_ch_T_acc_list = []
aver_max_ind_const_T_acc_list = []
aver_random_ind_const_T_acc_list = []

In [6]:
def train_with_special_idx(trainset, test_loader, scores, idx):
    teacher = ResNet18()
    teacher.load_state_dict(torch.load(f'resnet_breeds.pt'))
    teacher.to(device)
    freeze_model(teacher)
    
    # модель студента
    model = ResNet18()
    model.load_state_dict(torch.load(f'resnet18.pt'))
    model.eval()
    model.to(device)
    
    indexData = Breeds(trainset, scores, idx)
    idx_train_loader = torch.utils.data.DataLoader(indexData,**train_kwargs)
    return train_with_teacher(model, teacher, idx_train_loader, test_loader, IS_CONST_T)

In [ ]:
bs_list = [200, 1000, 2000, 3000, 4000, 5000]

num_repeat = 1
global IS_CONST_T
for bs in bs_list:
    print(bs)
    random_ind_ch_T_acc_list = []
    random_ind_const_T_acc_list = []
    for _ in range(num_repeat):
        random_indexes = np.random.choice(scores.argsort(), size=bs, replace=False)
    
        IS_CONST_T = True
        random_idx_loss_const_T, random_idx_acc_const_T = train_with_special_idx(trainset, test_loader, scores, random_indexes)
        
    
        IS_CONST_T = False
        random_idx_loss, random_idx_acc = train_with_special_idx(trainset, test_loader, scores, random_indexes) 
    
        random_ind_ch_T_acc_list.append(random_idx_acc[-1])
        random_ind_const_T_acc_list.append(random_idx_acc_const_T[-1])
        
    aver_random_ind_ch_T_acc_list.append(np.mean(random_ind_ch_T_acc_list))
    aver_random_ind_const_T_acc_list.append(np.mean(random_ind_const_T_acc_list))
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.plot(bs_list, random_ind_ch_T_acc_list, label = 'random_ind, T changed')
plt.plot(bs_list, random_ind_const_T_acc_list, label = 'random_ind, T const')


plt.title(f'Зависимость accuracy от размера подвыборки')
plt.xlabel('bs')
plt.ylabel('accuracy')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.plot(bs_list, max_ind_ch_T_acc_list, label = 'max_ind, T changed')
plt.plot(bs_list, max_ind_const_T_acc_list, label = 'max_ind, T const')


plt.title(f'Зависимость accuracy от размера подвыборки')
plt.xlabel('bs')
plt.ylabel('accuracy')
plt.legend(loc = 'lower right')
plt.show()